<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Task" data-toc-modified-id="Task-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Task</a></span></li><li><span><a href="#Method" data-toc-modified-id="Method-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Method</a></span></li><li><span><a href="#Approach:-Reasoning-process" data-toc-modified-id="Approach:-Reasoning-process-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Approach: Reasoning process</a></span><ul class="toc-item"><li><span><a href="#Step-2:" data-toc-modified-id="Step-2:-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Step 2:</a></span></li><li><span><a href="#step-3:" data-toc-modified-id="step-3:-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>step 3:</a></span></li><li><span><a href="#Step-4:" data-toc-modified-id="Step-4:-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Step 4:</a></span></li><li><span><a href="#Step-5:" data-toc-modified-id="Step-5:-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Step 5:</a></span></li><li><span><a href="#Last-Remarks" data-toc-modified-id="Last-Remarks-3.5"><span class="toc-item-num">3.5&nbsp;&nbsp;</span>Last Remarks</a></span></li></ul></li></ul></div>

## Task 

You are told that a Hill model was used with a 5 x 5 matrix key. You also know that PILLA is a cipher 
text that arose from the encryption of a common female name in English. Do everything you can 
through a code to brute force attack the Hill model and decode the name. Show every piece of your 
reasoning.

## Method
1. Google: Common 5 character English female names 
2. Create dataframe to store the common 5 character female names 
3. Generate 5x5 key matrices that have inverses (determinant is non-zero)
4. Using C = E(K, P) = P * K mod 26, where C is the generated ciphertext, K is the 5x5 key matrix and P is plaintext (common 5 character female names) encrypt the female names in the dataframe. 
5. Using the C generated from the encryption, Loop through the C characters and observe characters that match the characters of PILLA (e.g. if the first generated character from encryption matches P and so on). 

## Approach: Reasoning process

### Step 2:

Create a dataframe with female names that have 5 characters. Female names with less than 6 characters can also be generated since in the encryption process of `PILLA` could have filler characters. However to demonstrate the reasoning process 5 characters will be used to generate female names. Generating female names with less than 6 characters will consider female names that have 5, 4, 3, 2, 1 character(s).  

In [10]:
import names 
import pandas as pd
import numpy as np

In [21]:
name_list = []
for i in range(50): # for testing purposes 50 names will be generated
    rand_name = names.get_first_name(gender='female')
    if len(rand_name) == 5: # can also generate names with less than 6 characters to cater for other possibilities 
        name_list.append(rand_name)

In [23]:
# Create dataframe 
df = pd.DataFrame(name_list, columns=["Female_Names"])

In [26]:
df.head()

,Female_Names
0,Erika
1,Carol
2,Helen
3,Linda
4,Magda


### step 3:

Create a 5x5 matrix that have a inverse (determinant is non-zero)

In [33]:
for i in range(50): 
    key = np.random.randint(40, size=(5, 5)) # matrices that have values in between 0-40
    if np.linalg.det(key) != 0:
        break

In [34]:
key

array([[33, 31, 26, 14,  7],
       [17, 34,  6, 30, 38],
       [20, 17, 35, 32, 31],
       [ 9,  6, 31, 15, 16],
       [23,  0, 39, 13, 34]])

### Step 4:

Encrypt the female names using the generated matrix (Hill cipher method)

In [41]:
import string

def preprocessing(m):
    m = m.replace(" ","").replace(",","").replace(".","").replace("'","").replace(":","").replace(";","")
    m = m.lower()
    return m

def lettersOfPlaintext(m):
    letters = []
    for i in range(0, len(m)):
        letters.append(m[i])
    return letters

def letterToNumber(letter):
    return string.ascii_lowercase.index(letter)

def numberToLetter(number):
    return chr(int(number) + 97)

def module(letter_index):
    
    while(letter_index < 0):
        letter_index += 26
       
    while(letter_index > 25):
        letter_index -= 26
    
    return letter_index

In [108]:
# for demonstration the first female name is used. 
female_name = name_list[0]

In [109]:
female_name = preprocessing(female_name)
female_name

'erika'

In [110]:
plaintext = lettersOfPlaintext(female_name)

In [111]:
plaintext

['e', 'r', 'i', 'k', 'a']

In [112]:
# Obtaining the numeric representation of the female name characters 
plaintext_idx = []
for i in plaintext:
    plaintext_idx.append(letterToNumber(i))

In [113]:
plaintext_idx

[4, 17, 8, 10, 0]

In [114]:
plaintext_matrix = np.array(plaintext_idx)

In [115]:
plaintext_matrix

array([ 4, 17,  8, 10,  0])

In [116]:
plaintext_matrix.shape

(5,)

In [117]:
# encrypting the female name (Hill Cipher method)
encryption = np.matmul(plaintext_matrix, key) % 26

In [118]:
encryption.resize(1,5)

In [119]:
ciphertext = []
for i in encryption.tolist()[0]:
    ciphertext.append(numberToLetter(i))

In [120]:
# obtaining the ciphertext
ciphertext

['v', 'o', 'q', 'k', 'q']

In [121]:
ciphertext = ''.join(ciphertext)

In [103]:
ciphertext

'voqkq'

### Step 5:

Using the ciphertext generated from the encryption, Loop through the Ciphertext characters and observe characters that match the characters of PILLA (e.g. if the first generated character from encryption matches P and so on).

In [122]:
ciphername = 'PILLA'

In [124]:
import re 

# Ciphername: This is the ciphertext that we need to decipher
# ciphertext: This is the ciphertext generated to see if the encrypted female name generated the ciphername (PILLA)
# A function that returns that number of characters that matching between the cipher to decode and cipher generated 
def CipherMatch(ciphername, ciphertext):
    c = 0
    for i in str(ciphername.lower()):
        if re.search(i,ciphertext.lower()):
            c=c+1
    return c

In [125]:
c_match = CipherMatch(ciphername, ciphertext)

In [126]:
c_match

0

### Last Remarks

If the characters from `PILLA` match characters from the encrypted name, the female name that generated the encryption can be the output. This process can be done such that, there is loop to consider the different female names for encryption and matching the ciphertext from the female name encryption with `PILLA`

**Summary** <br>
- Create a dataframe with 5 character female names 
- Create 5x5 matrices that have inverses (determinant is non-zero)
- Encrypt the names in the dataframe 
- Check if those encrypted names have characters that match `PILLA`